<a href="https://colab.research.google.com/github/emrehangorgec/GANsNFTs/blob/main/Generative_Art_NFTs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install openai ninja  ipywidgets requests pinata

# API Configurations


In [ ]:
import getpass
import os


os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
is_api_key_set = 'OPENAI_API_KEY' in os.environ
print("Is the API key set?", is_api_key_set)
print("API Key:", os.environ["OPENAI_API_KEY"])

In [ ]:
pinata = Pinata(api_key=userdata.get('pinata-api-key'),
                secret_key=userdata.get('pinata-secret-key'),
                access_token=userdata.get('pinata-JWT'))

# Application of GANs

## Generate the content image with DALL-E

In [ ]:
import openai, requests
from IPython.display import Image, display
from ipywidgets import Button, Dropdown, IntSlider, Layout, Text, interact_manual

In [ ]:
def save_prompt(
    prompt,
    filename="/content/drive/MyDrive/GANsNFTs/prompts.txt",
):
    with open(filename, "a") as file:
        file.write(prompt + "\n")


def update_size_options(*args):
    if model_version.value == "dall-e-2":
        image_size.options = ["512x512", "256x256"]
    elif model_version.value == "dall-e-3":
        image_size.options = ["1024x1024", "1024x1792", "1792x1024"]
    else:
        image_size.options = ["1024x1024"]

In [ ]:
model_version = Dropdown(options=["dall-e-2", "dall-e-3"], description="Model:")
image_size = Dropdown(description="Size:")
image_quality = Dropdown(options=["standard", "high"], description="Quality:")
num_images = Dropdown(options=[1, 2, 3, 4, 5], description="Number:", value=1)
prompt_input = Text(
    value="", description="Prompt:", layout=Layout(width="800px", height="auto")
)

client = openai.OpenAI()

model_version.observe(update_size_options, "value")

In [ ]:
def generate_image(model, size, quality, n, prompt):
    save_prompt(prompt) 
    response = client.images.generate(
        model=model,
        prompt=prompt,
        size=size,
        quality=quality,
        n=n,
    )

    directory = "/content/drive/MyDrive/GANsNFTs/content"
    for i, item in enumerate(response.data):
        img_url = item.url
        display(Image(url=img_url))

        image_response = requests.get(img_url)
        file_count = len(
            [
                name
                for name in os.listdir(directory)
                if os.path.isfile(os.path.join(directory, name))
            ]
        )
        filename = f"image_{file_count}.jpg"
        save_path = os.path.join(directory, filename)
        with open(save_path, "wb") as f:
            f.write(image_response.content)

In [ ]:
update_size_options()

In [ ]:
ui = interact_manual(
    generate_image,
    model=model_version,
    size=image_size,
    quality=image_quality,
    n=num_images,
    prompt=prompt_input,
)
ui.widget.children[-2].description = "Generate Image"

## Generate the style image using StyleGAN3

In [ ]:
!python /content/drive/MyDrive/GANsNFTs/repos/stylegan3/gen_images.py --outdir=/content/drive/MyDrive/GANsNFTs/styles --trunc=1 --seeds=2,120,320,420 \
    --network=/content/drive/MyDrive/GANsNFTs/StyleGAN/gan/experiments/cubism_256x256/00002-stylegan3-t-Cubism-1000-256x256-gpus1-batch32-gamma6.6/network-snapshot-000200.pkl

## Neural Style Transfer

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [ ]:
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

In [ ]:
content_path = "/content/drive/MyDrive/GANsNFTs/content/image_6.jpg"

In [ ]:
style_path = "/content/drive/MyDrive/GANsNFTs/styles/seed0120.png"

In [ ]:
def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

In [ ]:
content_image = load_image(content_path)
style_image = load_image(style_path)

In [ ]:
print(f"{content_image.shape, style_image.shape}")

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(np.squeeze(content_image))
plt.title('Content Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(np.squeeze(style_image))
plt.title('Style Image')
plt.axis('off')

plt.show()

In [ ]:
stylized_image = model(tf.constant(content_image), tf.constant(style_image))[0]

In [ ]:
plt.imshow(np.squeeze(stylized_image))
plt.title('Stylized Image')
plt.axis('off')
plt.show()

In [ ]:
directory = '/content/drive/MyDrive/GANsNFTs/stylized_images'

In [ ]:
def get_next_filename(directory, base_filename="image", extension=".jpg"):
    i = 0
    while True:
        new_filename = f"{base_filename}_{i}{extension}"
        if not os.path.exists(os.path.join(directory, new_filename)):
            return new_filename
        i += 1

In [ ]:
stylized_image_np = stylized_image.numpy()

if np.max(stylized_image_np) <= 1.0:
    stylized_image_np = (stylized_image_np * 255).astype(np.uint8)

if len(stylized_image_np.shape) == 4:
    stylized_image_np = stylized_image_np[0]

new_filename = get_next_filename(directory)
stylized_image_bgr = cv2.cvtColor(stylized_image_np, cv2.COLOR_RGB2BGR)
cv2.imwrite(os.path.join(directory, new_filename), stylized_image_bgr)

# NFT

## Upload to IPFS

In [ ]:
import requests
from pinata import Pinata
from google.colab import userdata

In [ ]:
pinata = Pinata(api_key=userdata.get('pinata-api-key'),
                secret_key=userdata.get('pinata-secret-key'),
                access_token=userdata.get('pinata-JWT'))

In [ ]:
file_path = '/content/drive/MyDrive/GANsNFTs/stylized_images/image_0.jpg'
response = pinata.pin_file(file_path)
print(response)

In [ ]:
ipfs_hash = response['data']['IpfsHash']
print("IPFS Hash:", ipfs_hash)

In [ ]:
base_url = "https://bronze-worried-hornet-934.mypinata.cloud/ipfs/"
image_url = f"{base_url}{ipfs_hash}"
print("Image URL:", image_url)

## Mint NFTs

In [ ]:
collection_Id = "6d86d86f-1b4a-4c2e-a59b-90e7cc4cb049"
api_key = userdata.get('crossmint-api-key')
chain = "polygon-amoy"
env = "staging"
wallet_address = "wallet-address-placeholder"  # Replace with actual wallet address
recipient_address = f"{chain}:{wallet_address}"

In [ ]:
url = f"https://{env}.crossmint.com/api/2022-06-09/collections/{collection_Id}/nfts"
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "x-api-key": api_key,
}
payload = {
    "recipient": recipient_address,
    "metadata": {
        "name": "NFTRIO",
        "image": image_url,
        "description": "minted NFT for Rio",
    },
}

In [ ]:
response = requests.post(url, headers=headers, json=payload)
response_data = response.json()
print(response_data)